<a href="https://colab.research.google.com/github/naafey-aamer/xray_classifier/blob/main/Pneumonia_Classifier_EffiecientNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install efficientnet
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping
import efficientnet.tfkeras as efn

# Load the pre-trained ResNet-50 model (without the top fully connected layers)
# base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# replacing ResNet50 with EfficientNetB0
base_model = efn.EfficientNetB0(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 1.5 MB/s eta 0:00:00
16804768/16804768 [==============================] - 0s 0us/step


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Define data directories for train, validation, and test sets
train_data_dir = '/content/drive/MyDrive/xray_pneu/train'
val_data_dir = '/content/drive/MyDrive/xray_pneu/val'
test_data_dir = '/content/drive/MyDrive/xray_pneu/test'

# Set up data generators for train, validation, and test sets
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(256, 256),
    batch_size=128,
    class_mode='binary',
    classes=['NORMAL', 'PNEUMONIA']  # Classes based on subfolder names
)

val_generator = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=(256, 256),
    batch_size=4,
    class_mode='binary',
    classes=['NORMAL', 'PNEUMONIA']
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',
    classes=['NORMAL', 'PNEUMONIA']
)

Mounted at /content/drive
Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [ ]:
# print(train_generator[])

In [ ]:
# Unfreeze all layers
for layer in base_model.layers:
    layer.trainable = True

# print(len(base_model.layers))

# keep the bottom layers (first 220) frozen
for layer in base_model.layers[:220]:
    layer.trainable = False

# new dense layers for binary classification
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = tf.keras.layers.Dropout(0.65)(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.65)(x)
output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

# new model with the added layers
model = tf.keras.Model(inputs=base_model.input, outputs=output)
model.compile(
    loss='binary_crossentropy',
    optimizer=SGD(lr=0.001, momentum=0.9),
    metrics=['accuracy']
)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 stem_conv (Conv2D)             (None, 128, 128, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 stem_bn (BatchNormalization)   (None, 128, 128, 32  128         ['stem_conv[0][0]']              
                                )                                                             

In [ ]:
def lr_schedule(epoch, lr):
    # Reduce learning rate by 20% every 2 epochs
    if (epoch+1) % 2 == 0:
        lr *= 0.8
    return max(lr, 3e-4)

# Calculate class weights to handle the class imbalance (1342 NORMAL labels, and 3876 PNEUMONIA labels)
total_train_samples = 1342 + 3876
class_weights = {
    0: total_train_samples / (2 * 1342),   # Weight for 'NORMAL' class
    1: total_train_samples / (2 * 3876)    # Weight for 'PNEUMONIA' class
}

lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_schedule, verbose=1)
earlystop = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

# Fit the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=16,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    callbacks=[lr_callback, earlystop],
    class_weight=class_weights
)


Epoch 1: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 1/16
41/41 [==============================] - 1487s 36s/step - loss: 0.4314 - accuracy: 0.7853 - val_loss: 0.4824 - val_accuracy: 0.6250 - lr: 0.0100

Epoch 2: LearningRateScheduler setting learning rate to 0.007999999821186066.
Epoch 2/16
41/41 [==============================] - 631s 15s/step - loss: 0.2142 - accuracy: 0.9126 - val_loss: 0.3519 - val_accuracy: 0.8125 - lr: 0.0080

Epoch 3: LearningRateScheduler setting learning rate to 0.007999999448657036.
Epoch 3/16
41/41 [==============================] - 634s 15s/step - loss: 0.1755 - accuracy: 0.9264 - val_loss: 0.2736 - val_accuracy: 0.8750 - lr: 0.0080

Epoch 4: LearningRateScheduler setting learning rate to 0.006399999558925629.
Epoch 4/16
41/41 [==============================] - 635s 15s/step - loss: 0.1584 - accuracy: 0.9362 - val_loss: 0.2018 - val_accuracy: 0.9375 - lr: 0.0064

Epoch 5: LearningRateScheduler setting learning rate to 0.0063

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator, steps=len(test_generator))
print(f'Test accuracy: {test_accuracy}')

20/20 [==============================] - 252s 13s/step - loss: 0.3857 - accuracy: 0.8782
Test accuracy: 0.8782051205635071
